In [1]:
f ="unamb_sent_14_6.conllu"

In [2]:
from nltk.corpus.reader.conll import *
reader = ConllCorpusReader(root = '', fileids = [f], columntypes = ['words','pos', 'tree', 'chunk', 'ne', 'ignore'])

In [3]:
text=[]
for i in reader.iob_sents():
    for j in i:
        text.append(j[0].lower())

In [4]:
more_text=open("training.txt", encoding='utf-8').readlines()

In [5]:
clean_train=[]
i=1
while i<=len(more_text):
    clean_train.append(more_text[i])
    i+=3

In [43]:
from pymystem3 import Mystem
m=Mystem()

In [6]:
import nltk
clean_train_tokens=[]
for i in clean_train:
    if i[-2:]=='\n':
        s=i[:-2]+'.'
        for t in nltk.word_tokenize(s):
            print(t)
            clean_train_tokens.append(t)
    else:
        s=i+'.'
        for t in nltk.word_tokenize(s):
            clean_train_tokens.append(t)

In [7]:
all_tokens=clean_train_tokens+text

In [36]:
#for chars
all_bigrams={}
all_unigrams={}
for word in all_tokens:
    word=word.lower()
    word_bigrams=[word[k:k+2] for k in range(len(word)-1)]
    for i in word_bigrams:
        if i in all_bigrams.keys():
            all_bigrams[i]+=1
        else:
            all_bigrams[i]=0
    word_unigrams=[word[k:k+1] for k in range(len(word))]
    for i in word_unigrams:
        if i in all_unigrams.keys():
            all_unigrams[i]+=1
        else:
            all_unigrams[i]=1
            

from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()
ad.only_alphabet_chars(u"кириллический", "CYRILLIC")
l1=len(all_bigrams)
l2=len(all_unigrams)
char_bigram_probs={}
for i in all_bigrams.items():
    for k, v in all_unigrams.items():
        if ad.is_cyrillic(i[0]) and i[0].isdigit()==False and i[0][0] == k:
            char_bigram_probs[i[0][0]+i[0][1]]=(i[1]/l1)/(v/l2)

import datrie
trie = datrie.Trie('йцукенгшщзхъфывапролджэячсмитьбю -ё')
for k, v in char_bigram_probs.items():
    trie[k]=v
trie.save('lmodel_chars.trie')

In [ ]:
#for words
import re
import nltk
from nltk.collocations import *
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()
ad.only_alphabet_chars(u"кириллический", "CYRILLIC")
clean_tokens=[]
for i in all_tokens:
    if ad.is_cyrillic(i) and i.isdigit()==False and i!='-':
        clean_tokens.append(i)
finder = BigramCollocationFinder.from_words(clean_tokens)
finder.apply_word_filter(lambda w: w in ('.,:;?!-...!!!?!!?...'))
bigram_measures = nltk.collocations.BigramAssocMeasures()
scored = finder.score_ngrams(bigram_measures.raw_freq)
unigrams={}
for i in clean_tokens:
    unigrams[i]=clean_tokens.count(i)/len(clean_tokens)
bigram_probs={}
for i in scored:
    for k, v in unigrams.items():
        if i[0][0] == k:
            bigram_probs[i[0][0]+' '+i[0][1]]=i[1]/v
import datrie
trie = datrie.Trie('йцукенгшщзхъфывапролджэячсмитьбю -ё')
for k, v in bigram_probs.items():
    trie[k]=v

In [ ]:
trie.save('lmodel.trie')